This notebook will demonstrate a simple compile and deployment workflow using a ResNet50 pre-trained image recognizer with sample input images

First, we define all of our parameters that will be used in the notebook

In [ ]:
# Setup our Sagemaker configuration tunables
aws_region = 'us-east-1'
aws_s3_folder = "DEMO-Sagemaker-Edge"

# We will use the ResNet50 image recognition model via Keras framework
model_framework = 'keras'
model_name = 'demo-' + model_framework
model_basename = 'resnet50'
image_size = 224
image_list = []
packaged_model_name = model_framework + "-model"
packaged_model_version = "1.0"

# Input images directory in our notebook...
image_paths = './images'

# Our Pelion Edge Gateway is a Nvidia Jetson Xavier
target_device = 'jetson_xavier'

# Set our Pelion API Configuration Here
api_key = 'INSERT_YOUR_PELION_APPLICATION_KEY_HERE'
device_id = 'INSERT_YOUR_PELION_XAVIER_EDGE_GATEWAY_DEVICEID_HERE' # Pelion Device ID of our Sagemaker Edge Agent PT device under our Pelion Edge gateway
endpoint_api = 'api.' + aws_region + '.mbedcloud.com'              # This is optional and the default        

Next we import some additional packages into our python environment... including the pelion/sagemaker controller package

In [ ]:
# Lets install some image utilities
! pip install ipympl

# Core imports for the notebook
import numpy as np 
import pandas as pd
import os
import json
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

# We import these to help us specifically with our selected pre-trained model
from tensorflow.keras.applications.resnet50 import preprocess_input

# We'll also use time for waiting on predictions to complete...
import time

# we will also use some helpers from numpy...
from numpy import asarray
from numpy import save

# We also need to install the Pelion Sagemaker Controller API
! pip install pelion_sagemaker_controller

Next we import the core class for this notebook

In [ ]:
import example_notebook

Next we allocate our notebook class... this will init both Sagamaker as well as the Pelion Controller API

In [ ]:
my_notebook = example_notebook.MyNotebook(api_key, device_id, endpoint_api, aws_s3_folder)

First, lets import a pre-trained Mobilenet v2 model via TF Hub...

In [ ]:
# Import the model - we will choose the ResNet50 model pre-trained with imagenet weights...
resnet50_model = ResNet50(weights='imagenet')

# Lets dump the model details... 
print(resnet50_model.summary())

Next we prepare our input images and upload them to S3 as preprocessed images for ResNet50 from Keras...

In [ ]:
# collect and prepare our images to analyze
print("")
print("Collecting images from: " + str(image_paths))
my_image_list = [os.path.join(image_paths,filename) for filename in os.listdir(image_paths) if os.path.isfile(image_paths + '/' + filename)]
print("")
print("Input Image Files: " + json.dumps(my_image_list))

# preprocess_input() is ResNet50 specific
print("")
print("Pre-processing input image files...")
preprocessed_images_list = preprocess_input(my_notebook.read_and_prep_images(my_image_list, image_size, image_size))

# Display our initial images
print("Displaying Input Images:")
my_notebook.display_images(image_list)

# Neo Sagemaker likes NCHW. Our image set is in NHCW... so we must convert it...
print("")
print("Current input shape as NHCW: " + str(preprocessed_images_list.shape))
nchw_preprocessed_images_list = tf.transpose(preprocessed_images_list, [0, 3, 1, 2])
print("")
print("Transposed input shape to NCHW: " + str(nchw_preprocessed_images_list.shape))

# Save the preprocessed images_list as a single input file locally... 
input_data_filename = 'preprocessed_images_' + str(time.time()) + '.input'
input_data_filename_saved = input_data_filename + '.npy'

# save to a numpy-compatible file
save(input_data_filename, nchw_preprocessed_images_list)

# Upload the images in the list to S3
print("")
print('Uploading preprocessed input images to ' + my_notebook.iot_folder + " in S3 bucket " + my_notebook.bucket + ' as: ' + input_data_filename_saved + "...")
print("")
my_notebook.sess.upload_data(input_data_filename_saved, my_notebook.bucket, my_notebook.iot_folder)

Lets compile up our model, then package it on S3

In [ ]:
# Lets configure our shape with the number of images we intend on analyzing (Keras-specific: Resnet50 is pulled from Keras framework)
input_data_shape = '{"input_1":' + str(nchw_preprocessed_images_list.shape).replace('(','[').replace(')',']') + '}'
print("")
print("Input Data Shape as NCHW (Required by Sagemaker Neo for Keras/ResNet50): " + json.dumps(input_data_shape))

# Compile up for our target Pelion Edge Gateway platform type
print("")
print("Initiating Sagemaker Neo compile of " + model_name + "...")
job_name = my_notebook.compile_model(resnet50_model, target_device, model_basename, input_data_shape, model_framework)

# Package up and store the compiled model onto S3
print("")
print("Packaging up Neo-compiled model and placing in S3...")
model_package = my_notebook.package_model(packaged_model_name, packaged_model_version, job_name)

Next, we (re)load our model since we have just (re)compiled it and (re)packaged it...

In [ ]:
# (re)load the model...
print('Reloading Model: ' + model_name + " using package: " + model_package + '...')
print("")
my_notebook.pelion_api.pelion_reload_model(model_name,model_package)

# Poll every 5 sec to look for the reload() completion....
while True:
    print("Reloading " + model_name + "...")
    is_running = my_notebook.pelion_api.pelion_cmd_is_running("reloadModel");
    if is_running == False:
        print("")
        print('Reload Completed!')
        print("")
        break
    time.sleep(5)

# Get the loaded model(s) info...
reload_result = my_notebook.pelion_api.pelion_list_models();
if 'response' in reload_result and len(reload_result['response']) > 0:
    if 'name' in reload_result['response'][0]:
        print("")
        print("Currently Loaded Model(s):")
        print(reload_result)
else:
    print("Nodel: " + model_name + " did NOT load properly. Check sagemaker edge agent logs on GW.")

Now lets do a prediction. We will store the results (with a timestamp) back on S3 so that we can pull it back to our notebook...

In [ ]:
# Invoke the prediction with our input image data
input_data = 's3:///' + input_data_filename_saved
output_result = 's3:///' + model_basename + '-predicted.data'
print("Invoking Prediction on Pelion Edge with Sagemaker. Model: " + model_name + " Input: " + input_data + " Output: " + output_result)
print("")
my_notebook.pelion_api.pelion_predict(model_name, input_data, output_result)

# Poll every 5 sec to look for the predict() completion....
while True:
    print('Predicting...')
    is_running = my_notebook.pelion_api.pelion_cmd_is_running("predict");
    if is_running == False:
        print("")
        print('Prediction Completed!')
        print("")
        break
    time.sleep(5)
    
# Now get the prediction result
prediction_result = my_notebook.pelion_api.pelion_last_cmd_result();
if 'details' in prediction_result:
    if 'output' in prediction_result['details']:
        print("")
        print("Prediction Results:")
        print(prediction_result)

Next we display our results....

In [ ]:
# Prediction results tensor filename in our notebook
prediction_results_tensor_filename = model_basename + '-prediction-output.tensor'

# Copy the results back to our notebook
my_notebook.copy_results_to_notebook(prediction_result['details']['output'][0]['url'],prediction_results_tensor_filename)

# Read in the output tensor file, convert it, then decode our predictions and display our results...
file_size = os.path.getsize(prediction_results_tensor_filename)
print("Prediction Output File Size: " + str(file_size) + " bytes")
with open(prediction_results_tensor_filename, 'r') as file:
    # Load the JSON-based tensor from its file in our notebook... 
    json_tensor = json.loads(file.read())
    print("")
    print('Output JSON Tensor: Name: ' + json_tensor['name'] + " Type: " + str(json_tensor['type']) + " Shape: " + str(json_tensor['shape']))
    
    # Convert our JSON-based output tensor to a numpy float tensor
    np_float_tensor = my_notebook.json_tensor_to_numpy_float_tensor(json_tensor)
    
    print("")
    print("Decoding Prediction results...")
    most_likely_labels = my_notebook.decode_predictions(np_float_tensor, top=1, class_list_path='./model/imagenet_class_index.json')

    print("")
    print("Displaying prediction results...")
    my_notebook.display_images(image_list,most_likely_labels)

Done!  As a Data Scientist, I could iterate on re-training the model with additional input data, then recompile/deploy/predict to assess training. 